# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298568290284                   -0.85    4.2
  2   -8.300230505934       -2.78       -1.25    1.0
  3   -8.300448988442       -3.66       -1.89    3.2
  4   -8.300462319824       -4.88       -2.77    2.4
  5   -8.300464097977       -5.75       -3.06    4.9
  6   -8.300464362863       -6.58       -3.23    6.9
  7   -8.300464504649       -6.85       -3.37    5.2
  8   -8.300464578035       -7.13       -3.51    1.9
  9   -8.300464627974       -7.30       -3.71    1.1
 10   -8.300464638695       -7.97       -3.89    1.5
 11   -8.300464643659       -8.30       -4.22    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67172669516                   -0.70    6.6
  2   -16.67863656427       -2.16       -1.14    1.1
  3   -16.67923938269       -3.22       -1.86    2.9
  4   -16.67927794568       -4.41       -2.77    3.4
  5   -16.67928587908       -5.10       -3.20    5.6
  6   -16.67928621388       -6.48       -3.51    3.4
  7   -16.67928622280       -8.05       -3.97    1.0
  8   -16.67928622424       -8.84       -4.62    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32562660266                   -0.56    6.6
  2   -33.33293303759       -2.14       -0.99    1.0
  3   -33.33592432082       -2.52       -1.69    3.8
  4   -33.33682169495       -3.05       -2.27    4.4
  5   -33.33692451719       -3.99       -2.71   10.9
  6   -33.33694381749       -4.71       -3.68    3.9
  7   -33.33694387461       -7.24       -3.93    6.5
  8   -33.33694392595       -7.29       -4.92    2.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298597125729                   -0.85    4.2
  2   -8.300292414421       -2.77       -1.59    1.0
  3   -8.300451062586       -3.80       -2.57    7.1
  4   -8.300378935489   +   -4.14       -2.29   14.6
  5   -8.300464526840       -4.07       -3.73    2.9
  6   -8.300464597373       -7.15       -3.97    5.5
  7   -8.300464643650       -7.33       -4.95    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32570230280                   -0.56    6.1
  2   -33.30713703803   +   -1.73       -1.26    1.0
  3   -8.996660283016   +    1.39       -0.37    8.6
  4   -33.29832706666        1.39       -1.51    7.1
  5   -33.10705353669   +   -0.72       -1.28    4.6
  6   -27.97712684219   +    0.71       -0.70    5.1
  7   -33.33378414662        0.73       -2.10    7.5
  8   -33.33141992563   +   -2.63       -1.94    3.6
  9   -33.33187408407       -3.34       -2.07    3.5
 10   -33.33555771738       -2.43       -2.48    2.6
 11   -33.33669595457       -2.94       -2.79    2.9
 12   -33.33687360701       -3.75       -3.02    3.4
 13   -33.33687683341       -5.49       -3.07    1.1
 14   -33.33693809996       -4.21       -3.43    3.0
 15   -33.33694366761       -5.25       -4.17    4.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.